Bhavesh Bhatt

[**Link to my YouTube Channel**](https://www.youtube.com/BhaveshBhatt8791?sub_confirmation=1)

# Imports

In [2]:
import os
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import accuracy_score
from sys import getsizeof

/home/fridh/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
2025-03-17 22:24:06.305721: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742246646.422213   10973 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742246646.454728   10973 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-17 22:24:06.732939: I tensorflow/core/platform/cpu_feat

In [3]:
print(tf.__version__)

2.18.0


# Helper Functions

In [4]:
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

In [5]:
def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

# Importa Data

In [6]:
import  pandas as pd

df = pd.read_csv("../output_frames.csv", header=None)              # No header in your format
dataset = df.iloc[:, :-1].to_numpy(dtype=np.float16)            # All but last column as float16
labels_set = df.iloc[:, -1].to_numpy(dtype=str)                 # Last column as string

In [7]:
dataset = dataset.reshape(dataset.shape[0], 25, 16)
input_shape = dataset[0].shape

print(f"Dataset shape: {dataset.shape}")
print(f"Labels shape: {labels_set.shape}")
print(f"Input shape: {input_shape}")

Dataset shape: (583554, 25, 16)
Labels shape: (583554,)
Input shape: (25, 16)


In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(dataset, labels_set, test_size=0.2, random_state=42, stratify=labels_set)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# Print the lengths of the training, validation, and testing sets.
print(f"Training set length: {len(x_train)}")
print(f"Validation set length: {len(x_val)}")
print(f"Testing set length: {len(x_test)}")

Training set length: 373474
Validation set length: 93369
Testing set length: 116711


In [9]:
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights.
class_weight = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
dist_class_weight = dict(enumerate(class_weight))

In [10]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Hot end code the labels.
label_encoder = LabelEncoder()
y_train = to_categorical(label_encoder.fit_transform(y_train))
y_test = to_categorical(label_encoder.fit_transform(y_test))
y_val = to_categorical(label_encoder.fit_transform(y_val))

## Build & Compile the model


In [11]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import SeparableConv1D
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.regularizers import l2

model = keras.Sequential([
    InputLayer(shape=input_shape),  # 25 frames, 16 channels

    SeparableConv1D(filters=256, kernel_size=5, activation="swish", padding='same'),
    BatchNormalization(),
    SeparableConv1D(filters=512, kernel_size=5, activation="swish", padding='same'),
    BatchNormalization(),
    SeparableConv1D(filters=512, kernel_size=5, activation="swish", padding='same'),
    BatchNormalization(),
    SeparableConv1D(filters=1024, kernel_size=5, activation="swish", padding='same'),

    # Feature Pooling (Combining Max & Average Pooling)
    GlobalAveragePooling1D(),

    # Fully Connected Layer
    Dense(64, activation="swish", kernel_regularizer=l2(0.001)),
    Dropout(0.4),  # Dropout slightly reduced for stability

    # Output Layer (Softmax for multi-class classification)
    Dense(units=len(np.unique(labels_set)))
])

I0000 00:00:1742246680.901572   10973 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4623 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:09:00.0, compute capability: 7.5


In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ separable_conv1d                │ (None, 25, 256)        │         4,432 │
│ (SeparableConv1D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 25, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv1d_1              │ (None, 25, 512)        │       132,864 │
│ (SeparableConv1D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 25, 512)        │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv1d_2              │ (None, 25, 512)        │       265,216 │
│ (SeparableConv1D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 25, 512)        │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv1d_3              │ (None, 25, 1024)       │       527,872 │
│ (SeparableConv1D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 1024)           │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,001,429 (3.82 MB)

 Trainable params: 998,869 (3.81 MB)

 Non-trainable params: 2,560 (10.00 KB)

In [13]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['categorical_accuracy']
)

In [99]:
model.fit(x_train, y_train, epochs=64, validation_data=(x_val, y_val), class_weight=dist_class_weight)

Epoch 1/64
11672/11672 ━━━━━━━━━━━━━━━━━━━━ 74s 6ms/step - categorical_accuracy: 0.5550 - loss: 0.9592 - val_categorical_accuracy: 0.5680 - val_loss: 1.0472
Epoch 2/64
11672/11672 ━━━━━━━━━━━━━━━━━━━━ 73s 6ms/step - categorical_accuracy: 0.5883 - loss: 0.9105 - val_categorical_accuracy: 0.5785 - val_loss: 1.0361
Epoch 3/64
11672/11672 ━━━━━━━━━━━━━━━━━━━━ 73s 6ms/step - categorical_accuracy: 0.6143 - loss: 0.8683 - val_categorical_accuracy: 0.6036 - val_loss: 0.9890
Epoch 4/64
11672/11672 ━━━━━━━━━━━━━━━━━━━━ 74s 6ms/step - categorical_accuracy: 0.6399 - loss: 0.8299 - val_categorical_accuracy: 0.6242 - val_loss: 0.9541
Epoch 5/64
11672/11672 ━━━━━━━━━━━━━━━━━━━━ 74s 6ms/step - categorical_accuracy: 0.6609 - loss: 0.7948 - val_categorical_accuracy: 0.6312 - val_loss: 0.9532
Epoch 6/64
11672/11672 ━━━━━━━━━━━━━━━━━━━━ 74s 6ms/step - categorical_accuracy: 0.6825 - loss: 0.7570 - val_categorical_accuracy: 0.6320 - val_loss: 0.9504
Epoch 7/64
11672/11672 ━━━━━━━━━━━━━━━━━━━━ 73s 6ms/step -

In [100]:
KERAS_MODEL_NAME = "tf_model_fashion_mnist.h5"

In [101]:
model.save(KERAS_MODEL_NAME)

In [102]:
convert_bytes(get_file_size(KERAS_MODEL_NAME), "MB")

File size: 11.521 Megabytes


In [103]:
keras_model_size = get_file_size(KERAS_MODEL_NAME)

In [104]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
print('\nTest accuracy is {}%'.format(round(100*test_acc, 2)))

313/313 - 1s - 3ms/step - categorical_accuracy: 0.7164 - loss: 1.1487

Test accuracy is 71.64%


# TF Lite Model

In [105]:
TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"

In [106]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp498ey310/assets


INFO:tensorflow:Assets written to: /tmp/tmp498ey310/assets


Saved artifact at '/tmp/tmp498ey310'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 25, 16), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  123641038707296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038279024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038705360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038713280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038715568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038714336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038713632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038714512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038716624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123640996416496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123640996415792: Te

W0000 00:00:1742252067.939394   10973 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742252067.939414   10973 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-17 23:54:27.939650: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp498ey310
2025-03-17 23:54:27.942629: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-17 23:54:27.942650: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp498ey310
2025-03-17 23:54:27.970130: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-17 23:54:28.100727: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp498ey310
2025-03-17 23:54:28.145984: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 206337 microseconds.


In [107]:
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

4008844

In [108]:
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

File size: 3914.887 Kilobytes


In [109]:
tflite_file_size = get_file_size(TF_LITE_MODEL_FILE_NAME)

# Check Input Tensor Shape

In [110]:
interpreter = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [ 1 25 16]
Input Type: <class 'numpy.float32'>
Output Shape: [1 5]
Output Type: <class 'numpy.float32'>


# Resize Tensor Shape

In [111]:
x_test = x_test.astype(np.float32)[0:9999]
y_test = y_test.astype(np.float32)[0:9999]

In [112]:
interpreter.resize_tensor_input(input_details[0]['index'], x_test.shape)
interpreter.resize_tensor_input(output_details[0]['index'], y_test.shape)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [9999   25   16]
Input Type: <class 'numpy.float32'>
Output Shape: [9999    5]
Output Type: <class 'numpy.float32'>


In [113]:
x_test.dtype

dtype('float32')

In [114]:
test_imgs_numpy = np.array(x_test, dtype=np.float32)

In [115]:
test_imgs_numpy.dtype

dtype('float32')

In [116]:
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

Prediction results shape: (9999, 5)


In [117]:
# Convert y_test from one-hot encoded format to label format
y_test_labels = np.argmax(y_test, axis=1)
acc = accuracy_score(prediction_classes, y_test_labels)

In [118]:
print('Test accuracy TFLITE model is {}%'.format(round(100*acc, 2)))

Test accuracy TFLITE model is 71.64%


In [119]:
tflite_file_size/keras_model_size

0.33185377137638666

# TF Lite Model Float 16

In [120]:
TF_LITE_MODEL_FLOAT_16_FILE_NAME = "tf_lite_float_16_model.tflite"

In [121]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.DEFAULT]
tf_lite_converter.target_spec.supported_types = [tf.float16]
tflite_model = tf_lite_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpfwxux147/assets


INFO:tensorflow:Assets written to: /tmp/tmpfwxux147/assets


Saved artifact at '/tmp/tmpfwxux147'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 25, 16), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  123641038707296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038279024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038705360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038713280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038715568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038714336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038713632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038714512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038716624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123640996416496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123640996415792: Te

W0000 00:00:1742252087.192491   10973 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742252087.192510   10973 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-17 23:54:47.192743: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpfwxux147
2025-03-17 23:54:47.195364: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-17 23:54:47.195384: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpfwxux147
2025-03-17 23:54:47.216232: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-17 23:54:47.318100: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpfwxux147
2025-03-17 23:54:47.346150: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 153410 microseconds.


In [122]:
tflite_model_name = TF_LITE_MODEL_FLOAT_16_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

2012420

In [123]:
convert_bytes(get_file_size(TF_LITE_MODEL_FLOAT_16_FILE_NAME), "KB")

File size: 1965.254 Kilobytes


In [124]:
tflite_float_16_file_size = get_file_size(TF_LITE_MODEL_FLOAT_16_FILE_NAME)

In [125]:
tflite_float_16_file_size/keras_model_size

0.16658896345012877

In [126]:
tflite_float_16_file_size/tflite_file_size

0.5019950888585338

# TF Lite Size Quantized

In [127]:
TF_LITE_SIZE_QUANT_MODEL_FILE_NAME = "tf_lite_quant_model.tflite"

In [128]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tf_lite_converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = tf_lite_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpiie98yei/assets


INFO:tensorflow:Assets written to: /tmp/tmpiie98yei/assets


Saved artifact at '/tmp/tmpiie98yei'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 25, 16), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  123641038707296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038279024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038705360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038713280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038715568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038714336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038713632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038714512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123641038716624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123640996416496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  123640996415792: Te

W0000 00:00:1742252088.919843   10973 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742252088.919865   10973 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-17 23:54:48.920102: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpiie98yei
2025-03-17 23:54:48.922101: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-17 23:54:48.922118: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpiie98yei
2025-03-17 23:54:48.939087: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-17 23:54:49.027224: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpiie98yei
2025-03-17 23:54:49.051346: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 131248 microseconds.


In [129]:
tflite_model_name = TF_LITE_SIZE_QUANT_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

1072272

In [130]:
convert_bytes(get_file_size(TF_LITE_SIZE_QUANT_MODEL_FILE_NAME), "KB")

File size: 1047.141 Kilobytes


In [131]:
tflite_float_quant_file_size = get_file_size(TF_LITE_SIZE_QUANT_MODEL_FILE_NAME)

In [132]:
tflite_float_quant_file_size/keras_model_size

0.08876312152363645

In [133]:
tflite_float_quant_file_size/ tflite_float_16_file_size

0.5328271434392423

# Accuracy of the Quantized Model

# Check Input Tensor Shape

In [134]:
interpreter = tf.lite.Interpreter(model_path = TF_LITE_SIZE_QUANT_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [ 1 25 16]
Input Type: <class 'numpy.float32'>
Output Shape: [1 5]
Output Type: <class 'numpy.float32'>


# Resize Tensor Shape

In [135]:
interpreter.resize_tensor_input(input_details[0]['index'], x_test.shape)
interpreter.resize_tensor_input(output_details[0]['index'], y_test.shape)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [9999   25   16]
Input Type: <class 'numpy.float32'>
Output Shape: [9999    5]
Output Type: <class 'numpy.float32'>


In [136]:
x_test.dtype

dtype('float32')

In [137]:
test_imgs_numpy = np.array(x_test, dtype=np.float32)

In [138]:
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

Prediction results shape: (9999, 5)


In [139]:
# Convert y_test from one-hot encoded format to label format
y_test_labels = np.argmax(y_test, axis=1)
acc = accuracy_score(prediction_classes, y_test_labels)

In [140]:
print('Test accuracy TFLITE Quantized model is {}%'.format(round(100*acc, 2)))

Test accuracy TFLITE Quantized model is 71.16%
